In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet('Additional_Data/df_Votes_filtered_net.parquet')
df.head()

,ID_Posting,ID_CommunityIdentity,VoteNegative,VotePositive,CreatedAt,ID_GodotObject
0,1017441133,6397,False,True,2017-01-01 01:34:05.550,2000050076866
1,1017441133,85518,False,True,2017-01-01 01:12:15.370,2000050076866
2,1017441133,97871,False,True,2017-01-01 01:35:57.820,2000050076866
3,1017441133,121556,False,True,2017-01-01 02:56:03.357,2000050076866
4,1017441133,131596,False,True,2017-01-01 01:46:53.420,2000050076866


In [3]:
df1 = pd.read_parquet('Additional_Data/df_Postings_filtered_net.parquet')
df1.head()

,ID_Posting,ID_Posting_Parent,ID_GodotObject,ID_CommunityIdentity,createdAt,Comment_wo_controlchars_len,Headline_wo_controlchars_len
0,1017441133,NaN,2000050076866,572484,2017-01-01 00:50:39.700,63.0,0.0
1,1017441173,1.017441e+09,2000050076866,521988,2017-01-01 01:01:10.533,54.0,0.0
2,1017441185,1.017441e+09,2000050076866,225492,2017-01-01 01:06:13.480,19.0,0.0
3,1017441195,NaN,2000050076866,565184,2017-01-01 01:08:14.700,89.0,0.0
4,1017441198,1.017441e+09,2000050076866,565184,2017-01-01 01:09:01.690,0.0,33.0


In [4]:
df1 = df1[['ID_Posting','ID_CommunityIdentity']]
df1.columns = ['ID_Posting','ID_CommunityIdentity_Creator']

In [5]:
dfj = df.join(df1.set_index('ID_Posting'), on='ID_Posting',)

In [6]:
dfj = dfj[['ID_CommunityIdentity','ID_CommunityIdentity_Creator','VoteNegative','VotePositive']]

In [7]:
dfj.head()

,ID_CommunityIdentity,ID_CommunityIdentity_Creator,VoteNegative,VotePositive
0,6397,572484,False,True
1,85518,572484,False,True
2,97871,572484,False,True
3,121556,572484,False,True
4,131596,572484,False,True


In [8]:
dfj.columns = ['UserID','otherUserID','VoteNegative','VotePositive']

In [16]:
user_rec = dfj.groupby('otherUserID').sum()[['VoteNegative','VotePositive']].reset_index()
user_rec.columns = ['UserID','ReceivedNegative','ReceivedPositive']

In [14]:
user_voted = dfj.groupby('UserID').sum()[['VoteNegative','VotePositive']].reset_index()

In [19]:
user_data = user_rec.join(user_voted.set_index('UserID'),on='UserID',how='outer').fillna(0)

In [20]:
user_data.head()

,UserID,ReceivedNegative,ReceivedPositive,VoteNegative,VotePositive
0.0,4,0.0,6.0,5.0,50.0
1.0,30,117.0,288.0,10.0,194.0
2.0,38,3.0,59.0,1.0,5.0
3.0,77,285.0,486.0,0.0,0.0
NaN,106,0.0,0.0,0.0,1.0


In [21]:
user_data.to_parquet('Filtered_Data/user_user_votes.parquet',index=False)